In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

In [2]:
from fastkaggle import *

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'playground-series-s4e8'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('playground-series-s4e8')

In [6]:
from fastai.tabular.all import *

path.ls()

(#3) [Path('playground-series-s4e8/train.csv'),Path('playground-series-s4e8/test.csv'),Path('playground-series-s4e8/sample_submission.csv')]

In [7]:
df = pd.read_csv(path/'train.csv')
df['class'] = pd.Categorical(df['class'])

In [8]:
cats=['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']
conts=['cap-diameter', 'stem-height', 'stem-width']
dep='class'

In [9]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names=cats,
                   cont_names=conts,
                   y_names=dep,
                   splits=RandomSplitter(valid_pct=0.2)(range_of(df)))

/nix/store/dh9zjc820sfsz49pz3sld5x67wv3xa0c-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [10]:
to.xs.iloc[:2]

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,...,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,cap-diameter,stem-height,stem-width
1618016,72,0,62,9,48,0,50,0,0,50,...,0,6,19,0,37,1,1,-0.354199,-0.454914,-0.408132
2063380,72,0,58,9,53,0,60,0,0,56,...,22,19,20,0,26,1,1,1.135313,1.841204,0.640655


In [11]:
dls = to.dataloaders(bs=64)
dls.show_batch()

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,cap-diameter,stem-height,stem-width,class
0,x,t,n,f,s,#na#,g,#na#,y,n,#na#,#na#,f,f,#na#,d,w,False,6.27,5.44,6.770000,p
1,x,k,n,f,a,#na#,e,#na#,#na#,e,#na#,#na#,f,f,#na#,d,s,False,4.05,4.90,7.170000,p
2,x,t,e,f,x,d,w,b,#na#,w,#na#,#na#,f,f,#na#,d,u,False,7.72,5.41,16.860001,p
3,s,t,p,t,d,c,o,#na#,#na#,w,#na#,#na#,f,f,#na#,d,a,False,9.03,5.85,16.120001,e
4,x,e,y,t,p,#na#,o,#na#,#na#,u,#na#,#na#,f,f,#na#,d,a,False,10.87,9.01,30.520001,p
5,p,#na#,n,f,a,#na#,w,#na#,#na#,g,#na#,#na#,f,f,#na#,d,u,False,7.12,6.70,13.070000,e
6,x,h,y,f,s,#na#,p,#na#,i,n,#na#,#na#,f,f,#na#,d,a,False,12.97,6.66,16.910000,p
7,s,d,u,t,d,c,p,#na#,#na#,g,#na#,#na#,f,f,#na#,d,a,False,4.01,4.55,6.050000,e
8,s,h,g,t,d,#na#,w,#na#,t,g,#na#,#na#,f,f,#na#,d,u,False,8.24,4.72,12.810000,p
9,x,h,n,f,x,c,n,#na#,s,n,#na#,#na#,t,r,#na#,d,u,False,5.74,5.50,5.980000,e


In [14]:
from sklearn.metrics import matthews_corrcoef

learn = tabular_learner(dls, metrics=accuracy)

In [15]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.044116,0.037446,0.992104,06:05


In [16]:
row, clas, probs = learn.predict(df.iloc[0])
row.show()

/nix/store/dh9zjc820sfsz49pz3sld5x67wv3xa0c-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,cap-diameter,stem-height,stem-width,class
0,f,s,u,f,a,c,w,#na#,#na#,w,#na#,#na#,f,f,#na#,d,a,False,8.8,4.51,15.39,e


In [24]:
test_df = pd.read_csv(path/'test.csv')
stem_height_mode = test_df.mode().iloc[0]['stem-height']
test_df['stem-height'] = test_df['stem-height'].fillna(stem_height_mode)

test_dl = learn.dls.test_dl(test_df)
output = learn.get_preds(dl=test_dl, with_decoded=True)
probs,_,idxs = output

/nix/store/dh9zjc820sfsz49pz3sld5x67wv3xa0c-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [25]:
vocab = np.array(learn.dls.vocab)
results = pd.Series(vocab[idxs], name="class")
results

0          e
1          p
2          p
3          p
4          e
          ..
2077959    p
2077960    p
2077961    p
2077962    e
2077963    e
Name: class, Length: 2077964, dtype: object

In [26]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

,id,class
0,3116945,e
1,3116946,e
2,3116947,e
3,3116948,e
4,3116949,e
...,...,...
2077959,5194904,e
2077960,5194905,e
2077961,5194906,e
2077962,5194907,e


In [27]:
test_df['class'] = results

sub_df = test_df[['id','class']]
sub_df.to_csv(f'submission.csv', index=False)

In [28]:
!head submission.csv

id,class
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
3116950,e
3116951,e
3116952,p
3116953,p


In [29]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'poisonous-mushrooms-03', comp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19.8M/19.8M [00:45<00:00, 460kB/s]


In [115]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-01',
                  title='Poisonous Mushrooms Submission 01',
                  file='01.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-01
